In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
vod10 = pd.read_csv('../data/데이터스쿨3차_VOD_10월.csv', encoding='cp949')
vod10.head(2)

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,67055000,꼬리에꼬리를무는그날이야기 55회(22/12/01),TV 시사/교양,기타,4500,예고된 추락 - 성수대교 붕괴 참사. 1994년 10월 21일. 전 세계를 경악시킨...,"장도연,장현성,장성규",1:15,20231028153353
1,65443000,꼬리에꼬리를무는그날이야기 55회(22/12/01),TV 시사/교양,기타,4494,예고된 추락 - 성수대교 붕괴 참사. 1994년 10월 21일. 전 세계를 경악시킨...,"장도연,장현성,장성규",1:15,20231015171106


In [32]:
def preprocessing(data):
  df = data.copy()
  
  # disp_rtm 문자열을 분 단위로 변경
  def convert_runtime(runtime_str):
    # 입력값이 NaN이면 0 반환
    if pd.isna(runtime_str):
        return 0
    hours = int(runtime_str.split(':')[0])
    minutes = int(runtime_str.split(':')[1])
    total_minutes = hours * 60 + minutes
    return total_minutes

  df['disp_rtm'] = df['disp_rtm'].apply(convert_runtime)

  # 런타임 0분인 것 제거
  df = df[df['disp_rtm'] != 0]

  # 무삭제 제거
  df['asset_nm'] = df['asset_nm'].str.replace(r'무삭제판|무삭제', '', regex = True)
  
  # 예고편 제거
  df = df[~df['asset_nm'].str.contains(r'\(예고편\)|\(예고\)', regex=True)]
  
  # 예약구매, 사전구매 제거
  df = df[~df['asset_nm'].str.contains('예약구매|사전구매')]

  # 00회가 포함된 행은 런타임이 0또는 1이므로 제거
  df = df[~df['asset_nm'].str.contains(r'\b00회\b')]
  
  ## 괄호와 그 안의 내용 제거 
  df['asset_nm'] = df['asset_nm'].str.replace(r'\([^()]*\)', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\[[^\[\]]*\]', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\.\.\.', '', regex=True)
  df['asset_nm'] = df['asset_nm'].str.replace(r'\…', '', regex=True) # … 기호 제거
  df['asset_nm'] = df['asset_nm'].str.replace('-', " ")
  df['asset_nm'] = df['asset_nm'].str.rstrip('.')
  # df['asset_nm'] = df['asset_nm'].str.strip()

  # df3['series_nm'] = df3['asset_nm'].str.replace(r'\d+회$', '', regex=True)
  # df3['series_nm'] = df3['series_nm'].str.replace(r'\d+회\.', '', regex=True) 
  # df3['series_nm'] = df3['series_nm'].str.replace(r'\d+화$', regex = True)

  # use_tms 분 단위로 변경
  df['use_tms'] = round(df['use_tms'] / 60, 1)

  # 셋탑번호가 66056000인 것 삭제 - content 에서 이상치 아이디로 판별
  df = df[df['subsr'] != 66056000]

  return df

In [39]:
df = preprocessing(vod10)
df

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt
0,67055000,꼬리에꼬리를무는그날이야기 55회,TV 시사/교양,기타,75.0,예고된 추락 - 성수대교 붕괴 참사. 1994년 10월 21일. 전 세계를 경악시킨...,"장도연,장현성,장성규",75,20231028153353
1,65443000,꼬리에꼬리를무는그날이야기 55회,TV 시사/교양,기타,74.9,예고된 추락 - 성수대교 붕괴 참사. 1994년 10월 21일. 전 세계를 경악시킨...,"장도연,장현성,장성규",75,20231015171106
2,65443000,꼬리에꼬리를무는그날이야기 55회,TV 시사/교양,기타,35.4,예고된 추락 - 성수대교 붕괴 참사. 1994년 10월 21일. 전 세계를 경악시킨...,"장도연,장현성,장성규",75,20231015134804
3,65443000,꼬리에꼬리를무는그날이야기 55회,TV 시사/교양,기타,31.3,예고된 추락 - 성수대교 붕괴 참사. 1994년 10월 21일. 전 세계를 경악시킨...,"장도연,장현성,장성규",75,20231009171930
4,65443000,꼬리에꼬리를무는그날이야기 55회,TV 시사/교양,기타,0.0,예고된 추락 - 성수대교 붕괴 참사. 1994년 10월 21일. 전 세계를 경악시킨...,"장도연,장현성,장성규",75,20231009171929
...,...,...,...,...,...,...,...,...,...
4557,66900000,핑크퐁 호기의 우주동요 01회,키즈,학습,1.0,천왕성. 태양계의 일곱 번째 행성 장난꾸러기 천왕성! 얇은 고리와 함께 옆으로 누워...,"핑크퐁,호기",1,20231004130401
4558,65198000,더 놀다 잘래요,키즈,학습,6.8,"무지개 물고기 작가 마르쿠스 피스터의 또 다른 인기 그림책! 잠자기 싫고, 아빠와 ...",-,9,20231022092802
4559,66900000,핑크퐁 호기의 우주동요 02회,키즈,학습,0.2,해왕성. 태양에서 가장 먼 행성 해왕성! 태양은 과연 푸른 해왕성을 알고 있을까요?,"핑크퐁,호기",1,20231004130548
4560,65198000,달샤베트,키즈,학습,4.9,인기작가 백희나 그림책. 교보문고/YES24 베스트셀러! 자꾸만 뜨거워져 가는 지구...,-,5,20231022092300


In [91]:
# 프로그램 회차별 정보
df1 = df[['asset_nm', 'ct_cl', 'genre_of_ct_cl', 'ACTR_DISP', 'disp_rtm']].drop_duplicates().reset_index(drop = True)
df1

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,꼬리에꼬리를무는그날이야기 55회,TV 시사/교양,기타,"장도연,장현성,장성규",75
1,그것이알고싶다 1369회,TV 시사/교양,기타,김상중,78
2,꼬리에꼬리를무는그날이야기 99회,TV 시사/교양,기타,"장도연,장현성,장성규",76
3,그것이알고싶다 1367회,TV 시사/교양,기타,김상중,78
4,꼬리에꼬리를무는그날이야기 94회,TV 시사/교양,기타,"장도연,장현성,장성규",74
...,...,...,...,...,...
1943,핑크퐁 호기의 우주동요 01회,키즈,학습,"핑크퐁,호기",1
1944,더 놀다 잘래요,키즈,학습,-,9
1945,핑크퐁 호기의 우주동요 02회,키즈,학습,"핑크퐁,호기",1
1946,달샤베트,키즈,학습,-,5


In [92]:
# 유저마다 각 회차를 본 시간을 더함
df2 = pd.DataFrame(df.groupby(['subsr','asset_nm', 'ct_cl', 'genre_of_ct_cl', 'disp_rtm'])['use_tms'].sum()).reset_index()
df2 = df2[(df2['disp_rtm'] != 0) & (df2['use_tms'] != 0)]
df2

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms
0,59900000,7인의 탈출 08회,TV드라마,기타,87,48.4
1,59900000,7인의 탈출 09회,TV드라마,기타,67,48.9
2,59900000,꽃선비 열애사 18회,TV드라마,기타,62,3.2
3,59900000,연인 파트1 10회,TV드라마,기타,82,9.2
4,59900000,톡파원 25시 81회,TV 연예/오락,기타,83,83.0
...,...,...,...,...,...,...
2584,67170000,빨간 모자,키즈,기타,5,5.0
2585,67170000,태극기휘날리며,영화,액션/어드벤쳐,147,0.5
2586,67174000,가문의 영광: 리턴즈,영화,코미디,99,1.4
2587,67202000,검객,영화,액션/어드벤쳐,100,26.8


In [40]:
# 시청시간으로 유저가 이 회차를 시청했는지 여부를 결정
## 시청시간을 런타임으로 나눠 0.1 이 넘으면 시청한 것으로 간주
df2['watched'] = df2['use_tms'] / df2['disp_rtm']
df2['watched'] = df2['watched'].apply(lambda x : 1 if x >= 0.2 else 0)
df2.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms,watched
0,59900000,7인의 탈출 08회,TV드라마,기타,87,48.4,1
1,59900000,7인의 탈출 09회,TV드라마,기타,67,48.9,1
2,59900000,꽃선비 열애사 18회,TV드라마,기타,62,3.2,0
3,59900000,연인 파트1 10회,TV드라마,기타,82,9.2,0
4,59900000,톡파원 25시 81회,TV 연예/오락,기타,83,83.0,1


In [53]:
# 회차 제거
df11 = df1.copy()

df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+회\.', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.replace(r'\d+화$', '', regex=True)
df11['asset_nm'] = df11['asset_nm'].str.strip()
df11['asset_nm'] = df11['asset_nm'].str.rstrip('.')
df11

,asset_nm,ct_cl,genre_of_ct_cl,ACTR_DISP,disp_rtm
0,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",75
1,그것이알고싶다,TV 시사/교양,기타,김상중,78
2,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",76
3,그것이알고싶다,TV 시사/교양,기타,김상중,78
4,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,"장도연,장현성,장성규",74
...,...,...,...,...,...
1943,핑크퐁 호기의 우주동요,키즈,학습,"핑크퐁,호기",1
1944,더 놀다 잘래요,키즈,학습,-,9
1945,핑크퐁 호기의 우주동요,키즈,학습,"핑크퐁,호기",1
1946,달샤베트,키즈,학습,-,5


In [55]:
# 유저 시청 정보의 회차 제거한 변수 추가
df2['series_nm'] = df2['asset_nm'].str.replace(r'\d+회$', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.replace(r'\d+회\.', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.replace(r'\d+화$', '', regex=True)
df2['series_nm'] = df2['series_nm'].str.strip()
df2['series_nm'] = df2['series_nm'].str.rstrip('.')
df2.head()

,subsr,asset_nm,ct_cl,genre_of_ct_cl,disp_rtm,use_tms,watched,series_nm
0,59900000,7인의 탈출 08회,TV드라마,기타,87,48.4,1,7인의 탈출
1,59900000,7인의 탈출 09회,TV드라마,기타,67,48.9,1,7인의 탈출
2,59900000,꽃선비 열애사 18회,TV드라마,기타,62,3.2,0,꽃선비 열애사
3,59900000,연인 파트1 10회,TV드라마,기타,82,9.2,0,연인 파트1
4,59900000,톡파원 25시 81회,TV 연예/오락,기타,83,83.0,1,톡파원 25시


In [64]:
df3 = df2[['subsr', 'series_nm', 'ct_cl',	'genre_of_ct_cl', 'watched']].copy()
df3.head()

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched
0,59900000,7인의 탈출,TV드라마,기타,1
1,59900000,7인의 탈출,TV드라마,기타,1
2,59900000,꽃선비 열애사,TV드라마,기타,0
3,59900000,연인 파트1,TV드라마,기타,0
4,59900000,톡파원 25시,TV 연예/오락,기타,1


In [65]:
# 유저의 시리즈별 시청 횟수
df4 = df3.groupby(['subsr', 'series_nm', 'ct_cl', 'genre_of_ct_cl'])['watched'].sum().reset_index()
df4 = df4[df4['watched']!= 0]
df4

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched
0,59900000,7인의 탈출,TV드라마,기타,2
3,59900000,톡파원 25시,TV 연예/오락,기타,1
8,59930000,법쩐,TV드라마,기타,1
10,59933000,런닝맨,TV 연예/오락,기타,3
11,60050000,교전,영화,액션/어드벤쳐,1
...,...,...,...,...,...
1160,67170000,고딩엄빠 2,TV 연예/오락,기타,5
1162,67170000,말죽거리잔혹사,영화,액션/어드벤쳐,1
1163,67170000,빨간 모자,키즈,기타,1
1166,67202000,검객,영화,액션/어드벤쳐,1


In [69]:
# 시리즈별 시청된 총 횟수
df5 = pd.DataFrame(df11[['asset_nm', 'ct_cl', 'genre_of_ct_cl']].value_counts().reset_index())
df5.columns = ['series_nm', 'ct_cl', 'genre_of_ct_cl', 'watched_all']
df5

,series_nm,ct_cl,genre_of_ct_cl,watched_all
0,금이야 옥이야,TV드라마,기타,64
1,이산,TV드라마,기타,40
2,교양반아 : 햇살은 내 곁에,TV드라마,외화 시리즈,31
3,한문철의 블랙박스 리뷰,TV 시사/교양,인물/다큐,30
4,런닝맨,TV 연예/오락,기타,28
...,...,...,...,...
636,살수,영화,액션/어드벤쳐,1
637,살림하는 남자들 2,TV 연예/오락,기타,1
638,사자,영화,액션/어드벤쳐,1
639,사일런싱,영화,공포/스릴러,1


In [71]:
df6 = df4.merge(df5, on = ['series_nm', 'ct_cl', 'genre_of_ct_cl'], how = 'left')
df6

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all
0,59900000,7인의 탈출,TV드라마,기타,2,12
1,59900000,톡파원 25시,TV 연예/오락,기타,1,2
2,59930000,법쩐,TV드라마,기타,1,1
3,59933000,런닝맨,TV 연예/오락,기타,3,28
4,60050000,교전,영화,액션/어드벤쳐,1,1
...,...,...,...,...,...,...
768,67170000,고딩엄빠 2,TV 연예/오락,기타,5,5
769,67170000,말죽거리잔혹사,영화,액션/어드벤쳐,1,1
770,67170000,빨간 모자,키즈,기타,1,1
771,67202000,검객,영화,액션/어드벤쳐,1,1


In [73]:
# 유저의 프로그램 총 시청 횟수
df7 = df6.groupby(['subsr'])['watched'].sum().reset_index()
df7.columns = ['subsr', 'watched_cnt']
df7

,subsr,watched_cnt
0,59900000,3
1,59930000,1
2,59933000,3
3,60050000,24
4,60067000,63
...,...,...
215,67133000,1
216,67164000,4
217,67170000,8
218,67202000,1


In [74]:
df8 = df6.merge(df7, on = 'subsr', how = 'left')
df8 = df8[df8['watched_cnt'] != 0].reset_index(drop = True)
df8

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all,watched_cnt
0,59900000,7인의 탈출,TV드라마,기타,2,12,3
1,59900000,톡파원 25시,TV 연예/오락,기타,1,2,3
2,59930000,법쩐,TV드라마,기타,1,1,1
3,59933000,런닝맨,TV 연예/오락,기타,3,28,3
4,60050000,교전,영화,액션/어드벤쳐,1,1,24
...,...,...,...,...,...,...,...
768,67170000,고딩엄빠 2,TV 연예/오락,기타,5,5,8
769,67170000,말죽거리잔혹사,영화,액션/어드벤쳐,1,1,8
770,67170000,빨간 모자,키즈,기타,1,1,8
771,67202000,검객,영화,액션/어드벤쳐,1,1,1


In [75]:
# 시청 여부, 해당 시리즈 총 횟수, 유저의 프로그램 총 시청 횟수를 이용해
# 유저의 프로그램 선호도를 측정
import numpy as np
def scoring(df):
  N = df['watched_all']
  L = df['watched_cnt']
  n = df['watched']
  lam = np.log(2) / 2
  w1 = 1 - np.exp(-1 * lam * N)
  data = pd.DataFrame([N, L]).T
  def custom_weight(data):
    if data['watched_cnt'] < data['watched_all']:
      return data['watched_all'] / data['watched_cnt']
    else:
      return 1

  w2 = data.apply(custom_weight, axis = 1)

  score = (n / N) * w1 * w2

  return score

In [76]:
df8['score'] = scoring(df8)
df8

,subsr,series_nm,ct_cl,genre_of_ct_cl,watched,watched_all,watched_cnt,score
0,59900000,7인의 탈출,TV드라마,기타,2,12,3,0.656250
1,59900000,톡파원 25시,TV 연예/오락,기타,1,2,3,0.250000
2,59930000,법쩐,TV드라마,기타,1,1,1,0.292893
3,59933000,런닝맨,TV 연예/오락,기타,3,28,3,0.999939
4,60050000,교전,영화,액션/어드벤쳐,1,1,24,0.292893
...,...,...,...,...,...,...,...,...
768,67170000,고딩엄빠 2,TV 연예/오락,기타,5,5,8,0.823223
769,67170000,말죽거리잔혹사,영화,액션/어드벤쳐,1,1,8,0.292893
770,67170000,빨간 모자,키즈,기타,1,1,8,0.292893
771,67202000,검객,영화,액션/어드벤쳐,1,1,1,0.292893
